# % NAV

General Notes:
- Equities where Strategy == Core should be classified based on the Core Sub Industry
column.
- The privates subtotal is simply where UD Sec Type == Privates
- Pharmaceuticals is the subtotal of Large Cap Pharma, Specialty Pharma, and Generics
- Biotechnology is the subtotal of Large Cap Biotech and Emerging Biotech
- Use any preferred method or library to process the data into a table with the specified
information.

In [39]:
import pandas as pd

In [40]:
raw_data = pd.read_excel('../../data/raw/Raw Exposure Data.xlsx', sheet_name='Grid Records')
raw_data.head(10)

,Date,Security Ticker,Lookthrough Tag,% Benchmark Weight,% NAV,Strategy,Core Sub Industry,UD Sec Type
0,2024-03-31,NaN,Core,NaN,0.000000,CASH,Cash,Cash
1,2024-03-31,NaN,Core,NaN,0.000300,CASH,Cash,Cash
2,2024-03-31,NaN,Core,NaN,0.033100,CASH,Cash,Cash
3,2024-03-31,NaN,Core,NaN,0.016700,CASH,Cash,Cash
4,2024-03-31,NaN,Core,NaN,0.000000,CASH,Cash,Cash
5,2024-03-31,NaN,Core,NaN,0.000700,CASH,Cash,Cash
6,2024-03-31,NaN,Core,NaN,0.047000,CASH,Cash,Cash
7,2024-03-31,NaN,Core,NaN,-10.407100,CASH,Cash,Cash
8,2024-03-31,NaN,Core,NaN,1.541059,CORE,Emerging Biotech,Equity
9,2024-03-31,NaN,Core,NaN,2.492559,CORE,Japan,Equity


In [41]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                263 non-null    datetime64[ns]
 1   Security Ticker     0 non-null      float64       
 2   Lookthrough Tag     263 non-null    object        
 3   % Benchmark Weight  136 non-null    float64       
 4   % NAV               127 non-null    float64       
 5   Strategy            127 non-null    object        
 6   Core Sub Industry   263 non-null    object        
 7   UD Sec Type         263 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 16.6+ KB


In [42]:
raw_data.describe()

,Date,Security Ticker,% Benchmark Weight,% NAV
count,263,0.0,136.000000,127.000000
mean,2024-03-31 00:00:00,NaN,0.735294,0.787401
min,2024-03-31 00:00:00,NaN,0.027737,-10.407100
25%,2024-03-31 00:00:00,NaN,0.133577,0.070150
50%,2024-03-31 00:00:00,NaN,0.254026,0.177500
75%,2024-03-31 00:00:00,NaN,0.745429,0.830733
max,2024-03-31 00:00:00,NaN,8.057823,9.247379
std,NaN,NaN,1.219239,1.879464


In [43]:
raw_data.describe(include='object')

,Lookthrough Tag,Strategy,Core Sub Industry,UD Sec Type
count,263,127,263,263
unique,3,2,11,3
top,M1WO0HC,CORE,Emerging Biotech,Equity
freq,136,119,77,242


- To create the desired table, we need to focus on the % NAV, Strategy, Core Sub Industry, and UD Sec Type columns. Based on the the requirements, we will only consider rows with non-null % NAV values (as % NAV is the primary metric) and rows where the Strategy is CORE (as other strategy categories are not reported in the table).
- We will begin by handling the rows with missing % NAV values, as this is the main reported metric. Additionally, we will only retain rows where Strategy equals CORE.

In [44]:
# Drop rows with missing '% NAV'
filtered_data = raw_data.dropna(subset=['% NAV'])
# Filter for 'CORE' strategy only
core_strategy_data = filtered_data[filtered_data['Strategy'] == 'CORE']

# Select relevant columns
core_strategy_data = core_strategy_data[['% NAV', 'Core Sub Industry', 'UD Sec Type']]

core_strategy_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 8 to 126
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   % NAV              119 non-null    float64
 1   Core Sub Industry  119 non-null    object 
 2   UD Sec Type        119 non-null    object 
dtypes: float64(1), object(2)
memory usage: 3.7+ KB


- There are 119 usable rows for analysis. We can further divide the data based on the 'Equity' and 'Privates' UD Sec Types. This separation is necessary because the sums for these types are calculated separately.
- Next, we calculate the % NAV for each Core Sub Industry group. Some renaming will also be done to match the values used in the sample table.

In [45]:
# Separate equity and private data
equity_data = core_strategy_data[core_strategy_data['UD Sec Type'] == 'Equity']
private_data = core_strategy_data[core_strategy_data['UD Sec Type'] == 'Privates']

# Group by 'Core Sub Industry' and sum '% NAV'
industry_sums = equity_data.groupby('Core Sub Industry', as_index=False)['% NAV'].sum()

# Replace specific values in 'Core Sub Industry'
industry_sums.replace({
    'Big Biotech': 'Large Cap Biotech',
    'Big Pharma': 'Large Cap Pharmaceuticals',
    'Spec Pharma': 'Specialty Pharmaceuticals'
}, inplace=True)

# If Generics category is not present, create a default sum value of 0. This is to avoid mismatch when calculating the delta later
if 'Generics' not in industry_sums['Core Sub Industry'].values:
    generics_row = pd.DataFrame({'Core Sub Industry': ['Generics'], '% NAV': [0]})
    industry_sums = pd.concat([industry_sums, generics_row], ignore_index=True)

From this data, we can separately calculate the total % NAV for the Pharmaceuticals sub-categories, Biotechnology sub-categories, and the Privates subtotal (as required in the table). Using these values, we can then calculate the Total Equity + Private Exposure. 

In [46]:
# Define sub-industry categories
pharmaceuticals_categories = ['Large Cap Pharmaceuticals', 'Specialty Pharmaceuticals', 'Generics']
biotechnology_categories = ['Large Cap Biotech', 'Emerging Biotech']

# Calculate subtotals for Pharmaceuticals and Biotechnology
pharmaceuticals_total = industry_sums.loc[
    industry_sums['Core Sub Industry'].isin(pharmaceuticals_categories), '% NAV'].sum()

biotechnology_total = industry_sums.loc[
    industry_sums['Core Sub Industry'].isin(biotechnology_categories), '% NAV'].sum()

# Calculate the subtotal for Privates
privates_total = private_data['% NAV'].sum()

# Avoid calculate bio and pharma categories twice
excluded_categories = pharmaceuticals_categories + biotechnology_categories
other_industries_total = industry_sums.loc[~industry_sums['Core Sub Industry'].isin(excluded_categories), '% NAV'].sum()

total_exposure = pharmaceuticals_total + biotechnology_total + privates_total + other_industries_total

# Create new rows for the subtotals
subtotal_rows = pd.DataFrame({
    'Core Sub Industry': ['Pharmaceuticals', 'Biotechnology', 'Privates', 'Total Equity + Private Exposure'],
    '% NAV': [pharmaceuticals_total, biotechnology_total, privates_total, total_exposure]
})

# Append the new rows to the existing dataframe
updated_data = pd.concat([industry_sums, subtotal_rows], ignore_index=True)
updated_data

,Core Sub Industry,% NAV
0,Large Cap Biotech,6.025287
1,Large Cap Pharmaceuticals,29.829038
2,Emerging Biotech,22.427696
3,Emerging Market,3.790794
4,Health Care Services,10.164629
5,Japan,6.229157
6,Life Sciences Tools,6.541752
7,Medtech/Devices,17.818169
8,Specialty Pharmaceuticals,1.073712
9,Generics,0.000000


Since I don't have access to the MSCI World Healthcare data, I will use hard-coded values from the example table to generate the MSCI World Healthcare and Delta columns.

In [47]:
# MSCI World Healthcare data
msci_world_healthcare = {
    'Biotechnology': 8.4,
    'Emerging Market': 0,
    'Health Care Services': 15.1,
    'Japan': 3.8,
    'Life Sciences Tools': 11.1,
    'Large Cap Biotech': 6.2,
    'Emerging Biotech': 2.1,
    'Large Cap Pharmaceuticals': 41.7,
    'Specialty Pharmaceuticals': 2.9,
    'Medtech/Devices': 16.9,
    'Privates': 0.0,
    'Generics': 0.2,
    'Pharmaceuticals': 44.8,
    # The total for the provided data is 100.1 not 100, but I followed the described data
    'Total Equity + Private Exposure': 100
}

# Round the data to math the format of hardcoded data before calculation
updated_data = updated_data.round(1)

# Map MSCI values to the updated dataframe and calculate the Delta
updated_data['MSCI World Healthcare'] = updated_data['Core Sub Industry'].map(msci_world_healthcare)
updated_data['Delta'] = updated_data['% NAV'] - updated_data['MSCI World Healthcare']

updated_data

,Core Sub Industry,% NAV,MSCI World Healthcare,Delta
0,Large Cap Biotech,6.0,6.2,-0.2
1,Large Cap Pharmaceuticals,29.8,41.7,-11.9
2,Emerging Biotech,22.4,2.1,20.3
3,Emerging Market,3.8,0.0,3.8
4,Health Care Services,10.2,15.1,-4.9
5,Japan,6.2,3.8,2.4
6,Life Sciences Tools,6.5,11.1,-4.6
7,Medtech/Devices,17.8,16.9,0.9
8,Specialty Pharmaceuticals,1.1,2.9,-1.8
9,Generics,0.0,0.2,-0.2
